### Importing Useful Libraries

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.nn.functional import softmax
import torch.nn.init as init

### Device Configuration

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Image Preprocessing 
### Normalizing Transforms

In [ ]:
normalize_transform = transforms.Normalize(
        mean=torch.tensor([125.29552899, 122.99125831, 113.90624687]) / 256,
        std=torch.tensor([62.9836127, 62.04402182, 66.63918649]) / 256
    )
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor(),
    normalize_transform])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    normalize_transform])

### Datasets and DataLoader
Default setup on CIFAR10 dataset, for running on CIFAR100 change CIFAR10 to CIFAR100 on train_dataset and test_dataset

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transform_test)


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=128, 
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=128, 
                                          shuffle=False)

### Defining Resnets
Implemented Resnet structures namely Resnet2, Resnet6, Resnet10, Resnet14, Resnet20 and Resnet56

In [ ]:

def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)
    
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels,track_running_stats=False)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels,track_running_stats=False)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, layers, block = ResidualBlock, num_classes=256):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16,track_running_stats=False)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes, bias = True)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels,track_running_stats=False))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
            out = self.conv(x)
            out = self.bn(out)
            out = self.relu(out)
            out = self.layer1(out)
            out = self.layer2(out)
            out = self.layer3(out)
            out = self.avg_pool(out)
            out = out.view(out.size(0), -1)
            out = self.fc(out)
            return out

class ResNet2(nn.Module):
    def __init__(self, block = ResidualBlock, num_classes=256):
        super(ResNet2, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16,track_running_stats=False)
        self.relu = nn.ReLU(inplace=True)
        self.avg_pool = nn.AvgPool2d(32)
        self.fc = nn.Linear(16, num_classes, bias = True)
        self.apply(_weights_init)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels,track_running_stats=False))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
            out = self.conv(x)
            out = self.bn(out)
            out = self.relu(out)
            out = self.avg_pool(out)
            out = out.view(out.size(0), -1)
            out = self.fc(out)
            return out

class ResNet6(nn.Module):
    def __init__(self, block = ResidualBlock, num_classes=256):
        super(ResNet6, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16,track_running_stats=False)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, 1)
        self.layer2 = self.make_layer(block, 32, 1, 2)
        self.avg_pool = nn.AvgPool2d(16)
        self.fc = nn.Linear(32, num_classes, bias = True)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels,track_running_stats=False))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
            out = self.conv(x)
            out = self.bn(out)
            out = self.relu(out)
            out = self.layer1(out)
            out = self.layer2(out)
            out = self.avg_pool(out)
            out = out.view(out.size(0), -1)
            out = self.fc(out)
            return out

class ResNet10(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet10, self).__init__()
        self.ResNet = ResNet([2,1,1])
    def forward(self,x):
        out = self.ResNet(x)
        return out

class ResNet14(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet14, self).__init__()
        self.ResNet = ResNet([2,2,2])
    def forward(self,x):
        out = self.ResNet(x)
        return out

class ResNet20(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet20, self).__init__()
        self.ResNet = ResNet([3,3,3])
    def forward(self,x):
        out = self.ResNet(x)
        return out

class ResNet56(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet56, self).__init__()
        self.ResNet = ResNet([9,9,9])
    def forward(self,x):
        out = self.ResNet(x)
        return out

### Deterministic Networks
Implementation of baseline network.

In [ ]:
class DeterministicNetwork(nn.Module):
    def __init__(self, base, num_classes=10):
        super(DeterministicNetwork, self).__init__()
        self.base = base
        self.fc = nn.Linear(256,num_classes, bias = True)
    def forward(self,x):
        out = self.base(x)
        out = self.fc(out)
        return softmax(out, dim = 1)

### Generator Network
Implemented G<sub>e</sub>, G<sub>h</sub> from 1-D Resnet6 blocks and combined them for Generator network.

In [ ]:
def conv3x31d(in_channels, out_channels, stride=1):
    return nn.Conv1d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)
    
class ResidualBlock1d(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock1d, self).__init__()
        self.conv1 = conv3x31d(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm1d(out_channels,track_running_stats=False)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x31d(out_channels, out_channels)
        self.bn2 = nn.BatchNorm1d(out_channels,track_running_stats=False)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out
class Ge(nn.Module):
    def __init__(self, block = ResidualBlock1d , num_classes=256):
        super(Ge, self).__init__()
        self.in_channels = 16
        self.conv = conv3x31d(2, 16)
        self.bn = nn.BatchNorm1d(16,track_running_stats=False)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, 1)
        self.layer2 = self.make_layer(block, 32, 1, 2)
        self.avg_pool = nn.AvgPool1d(128)
        self.fc = nn.Linear(32, num_classes, bias = True)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x31d(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm1d(out_channels,track_running_stats=False))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

class Gh(nn.Module):
    def __init__(self, block = ResidualBlock1d , num_classes=10):
        super(Gh, self).__init__()
        self.in_channels = 16
        self.conv = conv3x31d(3, 16)
        self.bn = nn.BatchNorm1d(16,track_running_stats=False)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, 1)
        self.layer2 = self.make_layer(block, 32, 1, 2)
        self.avg_pool = nn.AvgPool1d(128)
        self.fc1 = nn.Linear(32, 256*num_classes, bias = True)
        self.fc2 = nn.Linear(32,num_classes, bias = True)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x31d(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm1d(out_channels,track_running_stats=False))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out1 = self.fc1(out)
        out2 = self.fc2(out)
        return out1, out2

class GeneratorNetwork(nn.Module):
    def __init__(self,num_classes=10):
        super(GeneratorNetwork, self).__init__()
        self.Ge = Ge()
        self.num_classes = num_classes
        self.Gh = Gh(num_classes=num_classes)
        self.fc = nn.Linear(num_classes,256,bias=True)
    def forward(self,y,z1,z2,num_samples = 1):
        y = self.fc(y).view(num_samples,1,256)
        temp = self.Ge(torch.cat((z1,y),axis = 1))
        temp = temp.unsqueeze(1).repeat(1, 1, 1)
        W,b = self.Gh(torch.cat((z2,temp,y),axis=1))
        return W.view(num_samples,256,self.num_classes),b.view(num_samples,1,self.num_classes)

class CFNN(nn.Module):
    def __init__(self, base,num_classes=10 ):
        super(CFNN, self).__init__()
        self.base = base
        self.generator = GeneratorNetwork(num_classes = num_classes)
        self.num_classes = num_classes
    
    def sampler(self, num_samples): 
        m = torch.distributions.categorical.Categorical(torch.zeros(self.num_classes) + 1/self.num_classes)
        x = m.sample([int(num_samples)])
        return nn.functional.one_hot(x,self.num_classes).type(torch.float32)

    def forward(self,x,num_samples = 1):
        out = self.base(x)
        W,b = self.generator(self.sampler(num_samples).to(device),torch.rand(num_samples,1,256).to(device),torch.rand(num_samples,1,256).to(device),num_samples)
        out = softmax(torch.matmul(out,W) - b, dim=2)
        out = torch.mean(out,axis = 0)
        return out


### Mean Weights
Mean weights ablation study.

In [ ]:
class mean_weights(nn.Module):
  def __init__(self, base, model, num_classes):
    super(mean_weights, self).__init__()
    self.generator = model
    self.base = base
    self.num_classes = num_classes

  def sampler(self, num_samples): 
        m = torch.distributions.categorical.Categorical(torch.zeros(self.num_classes) + 1/self.num_classes)
        x = m.sample([int(num_samples)])
        return nn.functional.one_hot(x,self.num_classes).type(torch.float32)

  def forward(self,x, num_samples = 25):
    out = self.base(x)
    mW,mb = self.generator(self.sampler(num_samples).to(device),torch.rand(num_samples,1,256).to(device),torch.rand(num_samples,1,256).to(device),num_samples)
    mW = torch.tensor(mW)
    mb = torch.tensor(mb)
    mW = torch.mean(mW,axis = 0)
    mb = torch.mean(mb,axis = 0)
    out = torch.matmul(out,mW) - mb
    return softmax(out, dim = 1)


### Frozen Net
Experimental Study.

In [ ]:
# Linear Layer
class frozen_net_resnet0(nn.Module):
    def __init__(self, base, frozen, num_classes=10 ):
        super(frozen_net_resnet0, self).__init__()
        self.base = base
        self.frozen_net = frozen
        self.num_classes = num_classes
    
    def forward(self,x):
        out = self.base(x)
        out = out.repeat(1,1,1)
        out = out.permute(1,0,2)
        out = self.frozen_net(out)
        out = softmax(out, dim=1)
        return out

class frozen_net_resnet6(nn.Module):
  def __init__(self, block = ResidualBlock1d , num_classes=10):
        super(frozen_net_resnet6, self).__init__()
        self.in_channels = 16
        self.conv = conv3x31d(1, 16)
        self.bn = nn.BatchNorm1d(16,track_running_stats=False)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, 1)
        self.layer2 = self.make_layer(block, 32, 1, 2)
        self.avg_pool = nn.AvgPool1d(128)
        self.fc = nn.Linear(32, num_classes, bias = True)

  def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x31d(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm1d(out_channels,track_running_stats=False))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

  def forward(self, x):
      out = self.conv(x)
      out = self.bn(out)
      out = self.relu(out)
      out = self.layer1(out)
      out = self.layer2(out)
      out = self.avg_pool(out)
      out = out.view(out.size(0), -1)
      out = self.fc(out)
      return out

class frozen_net_resnet2(nn.Module):
  def __init__(self, block = ResidualBlock1d , num_classes=10):
        super(frozen_net_resnet2, self).__init__()
        self.in_channels = 16
        self.conv = conv3x31d(1, 16)
        self.bn = nn.BatchNorm1d(16,track_running_stats=False)
        self.relu = nn.ReLU(inplace=True)
        self.avg_pool = nn.AvgPool1d(256)
        self.fc = nn.Linear(16, num_classes, bias = True)

  def forward(self, x):
      out = self.conv(x)
      out = self.bn(out)
      out = self.relu(out)
      out = self.avg_pool(out)
      out = out.view(out.size(0), -1)
      out = self.fc(out)
      return out

class frozen_net_resnet4(nn.Module):
  def __init__(self, block = ResidualBlock1d , num_classes=10):
        super(frozen_net_resnet4, self).__init__()
        self.in_channels = 16
        self.conv = conv3x31d(1, 16)
        self.bn = nn.BatchNorm1d(16,track_running_stats=False)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, 1)
        self.avg_pool = nn.AvgPool1d(256)
        self.fc = nn.Linear(16, num_classes, bias = True)

  def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x31d(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm1d(out_channels,track_running_stats=False))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

  def forward(self, x):
      out = self.conv(x)
      out = self.bn(out)
      out = self.relu(out)
      out = self.layer1(out)
      out = self.avg_pool(out)
      out = out.view(out.size(0), -1)
      out = self.fc(out)
      return out

### Experimental Study 
Similar to Frozen Net, increasing the depth of embedding layer, for repeating n times change `depth = n` in init argument, training will be similar to frozen net only `frozen_net_resnet0` will be replaced by `neural_network()`.

In [ ]:
class neural_network(nn.Module):
  def __init__(self, base, depth = 2, num_classes = 10):
    super(neural_network, self).__init__()
    self.base = base
    self.nn = self.Make_network(depth, num_channels = 256)
    self.fc = nn.Linear(256,num_classes,bias=True)

  def Make_network(self, depth, num_channels =  256): 
        layers = []
        for i in range(depth):
            layers.append(nn.Linear(num_channels,num_channels,bias=True))
            layers.append(nn.ReLU())
        return nn.Sequential(*layers)

  def forward(self,x):
    out = self.base(x)
    out = self.nn(out)
    out = self.fc(out)
    return softmax(out, dim = 1)

### Hyperparameters

In [3]:
lr = 0.001
momentum = 0.9
weight_decay = 0.005
num_epochs = 200
num_samples = 25

### Defining Loss and Learning Rate Schedulers
Defined `NLLLoss` and Cosine Learning-Rate scheduler

In [4]:
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.nn import NLLLoss

### Defining Model Training 

In [5]:
#Training of the model
def Training(model,samples = None):
    Loss = NLLLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr,momentum = 0.9,weight_decay=0.005)
    scheduler = CosineAnnealingWarmRestarts(optimizer,T_0 = num_epochs,eta_min = 0)
    # optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum = 0.9, weight_decay = 0.005)
    total_step = len(train_loader)
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            if samples:
                output = model(images,samples)
                loss = Loss(output, labels) 
            else:
                output = model(images) 
            loss = Loss(torch.log(output+1e-015), labels)

            # Backward and optimize
            optimizer.zero_grad() 
            loss.backward()
            optimizer.step()
            if (i+1) % 100 == 0:
                print (" Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        scheduler.step()


### Defining Model Loss

In [6]:
def Testing(model,samples=None):
    model.eval()
    mean_accuracy = 0
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            if samples:
                output = model(images, samples) 
            else:
                output = model(images)
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
    
    
    return 100*correct/total

### For defining Baseline Network define it as shown below
Default on CIFAR10 dataset, dataset can be changed from CIFAR10 to CIFAR100 by changing `10` to `100` in argument of DeterministicNetwork, and Resnet structure can be changed by changing suffix of Resnet6 to Resnet10(or others) 

In [ ]:
# Resnet 6 / CIFAR 10
baseline_resnet = DeterministicNetwork(ResNet6(), 10).to(device)

### For defining Hypernet define it as shown below
Dataset can be changed from CIFAR10 to CIFAR100 by changing `10` to `100` in argument of CFNN similarly, and Resnet structure can be changed by changing suffix of Resnet6 to Resnet10(or others) 

In [ ]:
# Resnet 6 / CIFAR 10
CFNN_resnet = CFNN(ResNet6(), 10).to(device)

### Training of Baseline Network can be done as shown below 

In [ ]:
Training(baseline_resnet)

### Training of HyperNet can be done as shown below 

In [ ]:
Training(CFNN_resnet)

### Testing of Baseline Network and HyperNet can be done as 

In [ ]:
Testing(baseline_resnet)
Testing(CFNN_resnet)

### For defining Frozen Network define it as shown below
For running on different Frozen Resnet change suffix of resnet {0(linear layer) ,4, 2, 6}

In [ ]:
frozen_net = frozen_net_resnet0(CFNN_resnet.base,frozen_net_resnet4()).to(device)

### Training of Frozen Network can be done as follows

In [ ]:
frozen_net.base.requires_grad = False
Training(frozen_net)

### Testing of Frozen Network can be done as follows

In [ ]:
Testing(frozen_net)

### For defining Mean Weights define it as below
For changing dataset from CIFAR10 to CIFAR100 change 10 to 100 in argument of mean_weights_Resnet

In [ ]:
mean_Resnet = CFNN_resnet.base
mean_weights_Resnet = mean_weights(mean_Resnet, CFNN_resnet,10).to(device)

### For testing Mean Weights Ablation Study

In [ ]:
Testing(mean_weights_Resnet)